🎧 LyricLens (MVP): Song meaning from metadata + audio features

This notebook prototypes a “Spotify-style” feature that explains a song’s meaning without using full lyrics.
Optionally, a user can paste 1–2 lyric lines for interpretation (processed in-session only, not stored).

In [21]:
!pip install pydantic

In [22]:
import json
import re
from pathlib import Path
from typing import Optional, Dict, Any, List
from pydantic import BaseModel, Field

In [23]:
DATA_PATH = Path("data") / "sample_tracks.json"

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Missing {DATA_PATH}. Create data/sample_tracks.json first."
    )

dataset = json.loads(DATA_PATH.read_text(encoding="utf-8"))
len(dataset), list(dataset.keys())[:3]

(1, ['3n3Ppam7vgaVa1iaRUc9Lp'])

In [24]:
def extract_spotify_track_id(url_or_uri: str) -> str:
    s = (url_or_uri or "").strip()

    m = re.match(r"^spotify:track:([A-Za-z0-9]{22})$", s)
    if m:
        return m.group(1)

    m = re.search(r"open\.spotify\.com/track/([A-Za-z0-9]{22})", s)
    if m:
        return m.group(1)

    if re.match(r"^[A-Za-z0-9]{22}$", s):
        return s

    raise ValueError("Could not parse a Spotify track ID from that input.")

In [25]:
def get_track_bundle(dataset: dict, track_id: str) -> dict:
    track_id = track_id.strip()
    for key in dataset.keys():
        if key.strip() == track_id:
            item = dataset[key]
            return {"track": item["track"], "audio_features": item.get("audio_features", {})}

    raise KeyError(
        f"Track ID '{track_id}' not found. Available IDs: {list(dataset.keys())}"
    )


In [26]:
class MeaningOutput(BaseModel):
    summary: str
    themes: List[str] = Field(default_factory=list)
    emotional_tone: Dict[str, Any] = Field(default_factory=dict)  # {"label": str, "confidence": float}
    listening_clues: List[str] = Field(default_factory=list)
    interpretation_limits: str = ""
    line_explanation: Optional[Dict[str, Any]] = None

In [27]:
def mood_profile_from_audio(audio: dict) -> List[str]:
    if not audio:
        return ["No audio features available."]

    valence = audio.get("valence")
    energy = audio.get("energy")
    tempo = audio.get("tempo")
    acousticness = audio.get("acousticness")
    danceability = audio.get("danceability")

    notes = []
    if valence is not None:
        notes.append("bright/positive" if valence >= 0.6 else "somber/reflective" if valence <= 0.4 else "mixed/ambiguous")
    if energy is not None:
        notes.append("high intensity" if energy >= 0.65 else "mellow" if energy <= 0.35 else "moderate energy")
    if tempo is not None:
        notes.append("fast-paced" if tempo >= 120 else "slow-to-mid tempo" if tempo <= 95 else "mid tempo")
    if acousticness is not None:
        notes.append("organic/intimate" if acousticness >= 0.6 else "produced/electronic" if acousticness <= 0.3 else "hybrid texture")
    if danceability is not None:
        notes.append("groove-forward" if danceability >= 0.65 else "less groove-centric" if danceability <= 0.35 else "moderate groove")

    return notes

In [28]:
def generate_meaning(track: dict, audio: dict, user_line: Optional[str] = None) -> MeaningOutput:
    notes = mood_profile_from_audio(audio)
    artists = ", ".join(track.get("artists", []))

    valence = audio.get("valence", 0.5)
    energy = audio.get("energy", 0.5)

    summary = (
        f"“{track.get('name')}” by {artists} comes across as {notes[0]} and "
        f"{notes[1] if len(notes)>1 else 'distinctive'}. "
        "This interpretation is based on the track’s sonic mood (audio features) and metadata, not full lyrics."
    )

    themes = ["longing", "reflection", "uncertainty"] if valence < 0.45 else ["release", "confidence", "momentum"]
    emotional_tone = {"label": "reflective" if valence < 0.45 else "uplifting", "confidence": 0.55}

    listening_clues = [
        f"Energy suggests a {'driven/urgent' if energy > 0.65 else 'laid-back' if energy < 0.35 else 'balanced'} delivery.",
        "Notice how the groove (danceability) affects how ‘forward’ the track feels.",
        "Pay attention to whether the texture feels intimate (acousticness) or glossy/produced."
    ]

    out = MeaningOutput(
        summary=summary,
        themes=themes,
        emotional_tone=emotional_tone,
        listening_clues=listening_clues,
        interpretation_limits="Based on metadata + audio features, not full lyrics. Meaning is subjective and can vary by listener."
    )

    if user_line and user_line.strip():
        out.line_explanation = {
            "line": user_line.strip(),
            "interpretations": [
                {"reading": "Metaphorical", "why": "Could be expressing an emotional idea rather than a literal event.", "confidence": 0.5},
                {"reading": "Literal", "why": "Could describe a concrete moment that anchors the song’s feeling.", "confidence": 0.4}
            ],
            "caveat": "Interpreted without full lyrical context."
        }

    return out

In [29]:
# Try one from your sample_tracks.json:
# https://open.spotify.com/track/3n3Ppam7vgaVa1iaRUc9Lp

spotify_input = "https://open.spotify.com/track/3n3Ppam7vgaVa1iaRUc9Lp"
track_id = extract_spotify_track_id(spotify_input)
bundle = get_track_bundle(dataset, track_id)

track = bundle["track"]
audio = bundle["audio_features"]

track, audio

({'id': '3n3Ppam7vgaVa1iaRUc9Lp',
  'name': 'Mr. Brightside',
  'artists': ['The Killers'],
  'album': 'Hot Fuss',
  'release_date': '2004-06-07',
  'popularity': 85},
 {'danceability': 0.352,
  'energy': 0.911,
  'valence': 0.236,
  'tempo': 148.0,
  'acousticness': 0.001,
  'instrumentalness': 0.0,
  'liveness': 0.105,
  'speechiness': 0.074})

In [30]:
meaning = generate_meaning(track, audio, user_line=None)
meaning.dict()

C:\Users\ailey\AppData\Local\Temp\ipykernel_25004\2106153238.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  meaning.dict()


{'summary': '“Mr. Brightside” by The Killers comes across as somber/reflective and high intensity. This interpretation is based on the track’s sonic mood (audio features) and metadata, not full lyrics.',
 'themes': ['longing', 'reflection', 'uncertainty'],
 'emotional_tone': {'label': 'reflective', 'confidence': 0.55},
 'listening_clues': ['Energy suggests a driven/urgent delivery.',
  'Notice how the groove (danceability) affects how ‘forward’ the track feels.',
  'Pay attention to whether the texture feels intimate (acousticness) or glossy/produced.'],
 'interpretation_limits': 'Based on metadata + audio features, not full lyrics. Meaning is subjective and can vary by listener.',
 'line_explanation': None}

In [31]:
print("TRACK:", track["name"], "—", ", ".join(track["artists"]))
print("\nSUMMARY:\n", meaning.summary)

print("\nTHEMES:", ", ".join(meaning.themes))
print("\nEMOTIONAL TONE:", meaning.emotional_tone)

print("\nLISTENING CLUES:")
for c in meaning.listening_clues:
    print("-", c)

print("\nLIMITS:", meaning.interpretation_limits)

TRACK: Mr. Brightside — The Killers

SUMMARY:
 “Mr. Brightside” by The Killers comes across as somber/reflective and high intensity. This interpretation is based on the track’s sonic mood (audio features) and metadata, not full lyrics.

THEMES: longing, reflection, uncertainty

EMOTIONAL TONE: {'label': 'reflective', 'confidence': 0.55}

LISTENING CLUES:
- Energy suggests a driven/urgent delivery.
- Notice how the groove (danceability) affects how ‘forward’ the track feels.
- Pay attention to whether the texture feels intimate (acousticness) or glossy/produced.

LIMITS: Based on metadata + audio features, not full lyrics. Meaning is subjective and can vary by listener.


In [32]:
import json
from pathlib import Path

DATA_PATH = Path("data") / "sample_tracks.json"

dataset = json.loads(DATA_PATH.read_text(encoding="utf-8"))
print("Loaded keys:", list(dataset.keys()))


Loaded keys: ['3n3Ppam7vgaVa1iaRUc9Lp']


In [33]:
from pathlib import Path
Path().resolve()


WindowsPath('C:/Users/ailey/LyricLens')

In [34]:
import os
from pathlib import Path

os.chdir(r"C:\Users\ailey\LyricLens")
print("Now working in:", Path().resolve())


Now working in: C:\Users\ailey\LyricLens


In [35]:
from pathlib import Path
import json

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

json_path = data_dir / "sample_tracks.json"

starter_data = {
    "3n3Ppam7vgaVa1iaRUc9Lp": {
        "track": {
            "id": "3n3Ppam7vgaVa1iaRUc9Lp",
            "name": "Mr. Brightside",
            "artists": ["The Killers"],
            "album": "Hot Fuss",
            "release_date": "2004-06-07",
            "popularity": 85
        },
        "audio_features": {
            "danceability": 0.352,
            "energy": 0.911,
            "valence": 0.236,
            "tempo": 148.0,
            "acousticness": 0.001,
            "instrumentalness": 0.0,
            "liveness": 0.105,
            "speechiness": 0.074
        }
    }
}

json_path.write_text(json.dumps(starter_data, indent=2), encoding="utf-8")
print("JSON written to:", json_path.resolve())


JSON written to: C:\Users\ailey\LyricLens\data\sample_tracks.json


In [36]:
import json
from pathlib import Path

p = Path("data/sample_tracks.json")
print("Exists:", p.exists())

dataset = json.loads(p.read_text())
print("Keys:", list(dataset.keys()))


Exists: True
Keys: ['3n3Ppam7vgaVa1iaRUc9Lp']


In [37]:
spotify_input = "https://open.spotify.com/track/3n3Ppam7vgaVa1iaRUc9Lp"
track_id = extract_spotify_track_id(spotify_input)
bundle = get_track_bundle(dataset, track_id)
bundle["track"]["name"]


'Mr. Brightside'